<a href="https://colab.research.google.com/github/ab-sa/Statistical-Machine-Learning/blob/main/Lecture12_logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
Credit = pd.read_csv('Credit.csv')
print('Dimension of the data: ' + str(Credit.shape))
Credit.head()

Dimension of the data: (400, 12)


,ID,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [3]:
Credit.describe()
Credit['Balance_bin'] = pd.cut(Credit.Balance,
                               bins=[-1,460,2000],labels=['0','1'])
print(Credit['Balance_bin'].value_counts(normalize=True))
Credit.head()

1    0.5
0    0.5
Name: Balance_bin, dtype: float64


,ID,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance,Balance_bin
0,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333,0
1,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903,1
2,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580,1
3,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964,1
4,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331,0


**Classification**: Fit a logistic regression model with Balance_bin as response:

In [37]:
# data: dropping the credit-card related features (Limit & Rating) to make the classification problem more challenging
X = pd.get_dummies(Credit.drop(['ID', 'Balance', 'Balance_bin', 'Limit', 'Rating'], axis=1))
y = Credit['Balance_bin']

# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# create model
model = linear_model.LogisticRegression()
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.618 (0.096)


Logistic regression model with regularization:

In [41]:
modelCV = linear_model.LogisticRegressionCV(cv=5, random_state=0, max_iter=1000).fit(X, y)
print('Best regularization parameter: ', modelCV.C_)

Best regularization parameter:  [0.0001]


In [42]:
model_best = linear_model.LogisticRegression(C=modelCV.C_[0], max_iter=1000).fit(X, y)
print(model_best.score(X, y))
y_pred = model_best.predict(X)
print('Confusion matrix with counts: ')
print(confusion_matrix(y, y_pred))
print('Confusion matrix with proportions: ')
print(confusion_matrix(y, y_pred, normalize='true'))

0.6425
Confusion matrix with counts: 
[[153  47]
 [ 96 104]]
Confusion matrix with proportions: 
[[0.765 0.235]
 [0.48  0.52 ]]
